## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-29-20-48-05 +0000,nyt,Ghislaine Maxwell Lawyer Says She Will Only Te...,https://www.nytimes.com/2025/07/29/us/politics...
1,2025-07-29-20-46-00 +0000,wsj,S&P 500 Breaks Winning Streak After U.S.-China...,https://www.wsj.com/finance/stocks/s-p-500-bre...
2,2025-07-29-20-46-00 +0000,wsj,The Dow Jones Industrial Average fell more tha...,https://www.wsj.com/finance/stocks/s-p-500-bre...
3,2025-07-29-20-43-00 +0000,wsj,President Trump’s four-day trip to Scotland co...,https://www.wsj.com/politics/policy/trump-in-s...
4,2025-07-29-20-40-55 +0000,latimes,Metro can move forward on subway car plans ahe...,https://www.latimes.com/california/story/2025-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2300/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,55
63,gaza,22
178,new,17
12,china,16
4,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
205,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...,115
87,2025-07-29-17-38-03 +0000,nypost,UN-backed experts warn of ‘worst-case scenario...,https://nypost.com/2025/07/29/world-news/un-ba...,107
58,2025-07-29-18-32-06 +0000,nypost,UK will recognize Palestinian state unless Isr...,https://nypost.com/2025/07/29/world-news/uk-wi...,104
84,2025-07-29-17-49-06 +0000,nypost,"Moscow taunts Trump by comparing him to Biden,...",https://nypost.com/2025/07/29/world-news/mosco...,103
24,2025-07-29-20-08-55 +0000,nyt,"Scottish Leader Talks Gaza, Tariffs and Ukrain...",https://www.nytimes.com/2025/07/29/us/politics...,103


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
205,115,2025-07-29-09-30-00 +0000,wsj,Capital Account: Trump has hit his tariff goal...,https://www.wsj.com/economy/trade/trumps-new-t...
58,104,2025-07-29-18-32-06 +0000,nypost,UK will recognize Palestinian state unless Isr...,https://nypost.com/2025/07/29/world-news/uk-wi...
131,60,2025-07-29-15-30-51 +0000,nypost,Union Pacific strikes $85B deal to buy rival N...,https://nypost.com/2025/07/29/business/union-p...
34,54,2025-07-29-19-51-38 +0000,nypost,NYC shooter Shane Tamura’s brain could be test...,https://nypost.com/2025/07/29/us-news/medical-...
200,39,2025-07-29-09-50-34 +0000,cbc,Gunman who killed 4 at NYC building was target...,https://www.cbc.ca/news/world/gunman-manhattan...
207,36,2025-07-29-09-22-09 +0000,bbc,Beijing floods kill 30 as China sees summer of...,https://www.bbc.com/news/articles/cg7j8x3xnrko...
222,35,2025-07-29-08-17-56 +0000,nypost,Sen. John Kennedy weighs in on Park Ave skyscr...,https://nypost.com/2025/07/29/us-news/sen-john...
206,32,2025-07-29-09-24-00 +0000,wsj,Europe’s Star Economy Picks Up Pace as Fresh T...,https://www.wsj.com/world/europe/europes-star-...
237,28,2025-07-29-03-00-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
121,28,2025-07-29-16-01-39 +0000,nypost,"House probes whether EU, Biden administration ...",https://nypost.com/2025/07/29/media/house-prob...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
